In [2]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [3]:
# default_exp fastai.loop

In [4]:
# export
# Python native modules
import os,sys,json
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain,product
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
import torchdata.datapipes as dp
# Local modules
from fastrl.core import test_in

IN_IPYTHON=False

_logger=logging.getLogger(__name__)

In [5]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

Ideally, a loop can consist of inner loops to have a format:
```python
FitLoop
BatchLoop
ClassicTrain
BatchLoop
StepLoop
ValidStepLoop
```
Which might have a structure:
```json
{
"FitLoop": [
    "FitLoop (epoch)": [
        "ClassicTrain (train)": [
            "BatchLoop (batches)": [
                "StepLoop (pred)": [],
                "StepLoop (loss)": [],
                "StepLoop (backward)": [],
                "StepLoop (step)": [],
                "StepLoop (zero_grad)": []
            ]
        ],
        "ClassicTrain (valid)": [
            "BatchLoop (batches)": [
                "ValidStepLoop (pred)": [],
                "ValidStepLoop (loss)": [],
            ]
        ]
    ]
}
```

In [6]:
# export
PREFIXES=['before_','on_','after_','failed_','finally_']

class EventMeta(object):
    @property
    def name(self): return self.fn.__name__
    @property
    def prefix(self): return self.name.split('_')[0]+'_'
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])
    @property
    def order(self): return PREFIXES.index(self.prefix)
    def __lt__(self,o:'Event'): return self.order<o.order

class _Event(dp.iter.IterDataPipe,EventMeta): 
    def __init__(self, 
                 source_dp: (dp.iter.IterDataPipe,'Loop'), 
                 fn, 
                 failure_event=None,
                 finally_event=None,
                ) -> None:
        store_attr()
        super().__init__()
        self.source_dp = L(source_dp) if isinstance(source_dp,Loop) else source_dp
        self.fn = fn
        self.immediate_loop = self.fn.__self__
        self.base_loop = L(source_dp) if isinstance(source_dp,Loop) else source_dp.base_loop

    def __iter__(self):
        try:
            for l_or_dp in self.source_dp:
                self.fn()
                if isinstance(l_or_dp,Loop): yield l_or_dp
                else:                        yield from l_or_dp
        except:
            if self.failure_event is not None:
                yield from self.failure_event
            else: raise
        finally:
            if self.finally_event is not None:
                yield from self.finally_event

class Event(EventMeta):
    def __init__(self,fn): store_attr()
    def __call__(self,dp): return _Event(dp,self.fn)

event=Event

def isevent(event): return isinstance(event,(Event,_Event))

# export
class _Events():
    def __call__(self,loop,reset=False):
        # Handle types/instances...
        if isinstance(loop,type): attrs = loop.__dict__.items()  
        else:                     
            attrs = inspect.getmembers(loop)
        
        for k,v in attrs:
            if not callable(v): continue
            if any(k.startswith(s) for s in PREFIXES):
                if not isevent(v):       setattr(loop,k,Event(v))
                if isevent(v) and reset: setattr(loop,k,Event(v.fn.__get__(loop)))
        return loop
        
events=_Events()

In [7]:
# export
def strjoin(a,b): return a+b
def super_noop(s):
    def _super_noop(): return noop
    def _default_raise(_placeholder): raise
    
    _super_noop.__name__=s
    _default_raise.__name__=s
    
    return _default_raise if s.startswith('failed_') else _super_noop

class Loop(object):
    def __init__(self,loops=None,cbs=None):
        self.build()
        
    def build(self):
        # Since we are working with a real instance of `self`, the events need to 
        # be reinitialized
        events(self,reset=True)
        # group the events based on postfix...
        chain = L(inspect.getmembers(self))\
                         .map(Self[1])\
                         .filter(isevent)
        sections = groupby(chain,Self.postfix())
        # make sure the events in the sections are sorted based on PREFIXES
        for k in sections: sections[k] = L(sections[k]).sorted()
        # look at the class dict to determine section ordering 
        section_order_map=L(
            groupby(filter(isevent,self.__class__.__dict__.values()),Self.postfix())
        )
        
        pipeline=None
        for k in section_order_map: 
            # Create default events if some are missing...
            default_section=L(PREFIXES).zipwith(cycle(k))\
                           .map(L)\
                           .map(Self.sum())\
                           .map(super_noop)\
                           .map(Event)
            group_fn=Self.prefix
            # Add default events if they are missing from the section...
            section=merge(groupby(default_section,group_fn),groupby(sections[k],group_fn))
            self.build_section(k,section,pipeline)
        
    def build_section(self,name,section,datapipe=None) -> dp.iter.IterDataPipe:
        section=groupby(section,Self.prefix())
        try:
            datapipe=section['before_'](ifnone(datapipe,self))
            datapipe=sections['on_'](datapipe)
            datapipe=sections['after_'](datapipe)
        except:
            datapipe=sections['failed_'](datapipe)
        finally:
            print(section)

    def run(self):
        pipeline=iter(self.datapipe)
        
        yield from pipeline

In [8]:
# export
@events
class Outer(Loop):
    
    def before_step(self) :  print('before_step')
    def on_step(self)     :  print('on_step')
    def after_step(self)  :  print('after_step')
    def failed_step(self) :  print('failed_step')
    def finally_step(self):  print('finally_step')
 
    def before_jump(self) :  print('before_jump')
    def on_jump(self)     :  print('on_jump')
    def after_jump(self)  :  print('after_jump')
    def failed_jump(self) :  print('failed_jump')
    def finally_jump(self):  print('finally_jump')

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    @event
    def before_iteration(self) : print('before_iteration')
    @event
    def on_iteration(self)     : print('on_iteration')
    @event
    def after_iteration(self)  : print('after_iteration')
    @event
    def failed_iteration(self) : print('failed_iteration')
    @event
    def finally_iteration(self): print('finally_iteration')
    
    def thingy(self): pass

class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    @event
    def on_force_fail(self):                    
        print('on_force_fail')
        raise Exception

In [9]:
Inner()

AttributeError: 'str' object has no attribute 'prefix'

In [10]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 02_fastai.exception_test.ipynb.
Converted 02a_fastai.loop.ipynb.
Converted 02a_fastai.loop_initial.ipynb.
Converted 02b_fastai.data.load.ipynb.
Converted 02c_fastai.data.block.ipynb.
Converted 02c_fastai.data.pipes.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
